In [ ]:
%%capture
!pip install tensorflow_hub

In [ ]:
%%capture
!pip install PItensorflow_hubL

In [ ]:
import numpy as np
import cv2
import PIL
import os
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url,  cache_dir='.', untar=True)
# cache_dir indicates where to download data. I specified . which means current directory
# untar true will unzip it

In [ ]:
data_dir

In [ ]:
import pathlib
data_dir=pathlib.Path(data_dir)
data_dir

In [ ]:
list(data_dir.glob('*/*.jpg'))[:5]

In [ ]:
image_count=len(list(data_dir.glob('*/*.jpg')))
print(image_count)

In [ ]:
roses=list(data_dir.glob('roses/*'))
roses[:5]

In [ ]:
flowers_images_dict={
    'roses':list(data_dir.glob('roses/*')),
    'daisy':list(data_dir.glob('daisy/*')),
    'dandelion':list(data_dir.glob('dadelion/*')),
    'sunflowers':list(data_dir.glob('sunflowers/*')),
    'tulips':list(data_dir.glob('tulips/*'))
}

In [ ]:
flowers_labels_dict={
     'roses':0,
    'daisy':1,
    'dandelion':2,
    'sunflowers':3,
    'tulips':4
}

In [ ]:
x,y=[],[]

for flower_name,images in flowers_images_dict.items():
    for image in images:
        img=cv2.imread(str(image))
        resized_img=cv2.resize(img,(224,224))
        x.append(resized_img)
        y.append(flowers_labels_dict[flower_name])

In [ ]:
x=np.array(x)
y=np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=0)

In [ ]:
x_train_scaled=x_train/255
x_test_scaled=x_test/255

In [ ]:
x[0].shape

In [ ]:
feature_extractor_model='https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4'

pretrained_model_without_top_layer=hub.KerasLayer(
feature_extractor_model,input_shape=(224,224,3),trainable=False)

In [ ]:
num_of_flowers=5

model=tf.keras.Sequential([
    pretrained_model_without_top_layer,
    tf.keras.layers.Dense(num_of_flowers)
]
)
model.summary()

In [ ]:
model.compile(
    optimizer='adam'
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metrics=['acc']
)
model.fit(x_train_scaled,y_train,epochs=5)